# 하이퍼파라미터 튜닝
* 모델 파라미터 : 머신러닝 모델이 학습하는 파라미터
* 하이퍼파라미터 : 사용자가 지정해야 하는 파라미터 (모델이 학습할 수 없음)

#### 최적의 하이퍼파라미터 찾기
* 먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련
* 그 다음 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 변경
* 이 매개변수를 변경하면서 모델을 훈련하고, 교차 검증을 수행 
* 사람의 개입없이 자동으로 하이퍼파라미터를 찾아주는 **AutoML** 사용 가능

#### GridSearchCV

* 사이킷런에서 제공하는 그리드 서치(Grid Search)를 사용하여 하이퍼파라미터 찾기 가능 
* 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행 
    * : corss_validate() 함수를 따로 호출할 필요 **X**
* min_impurity_decrease : '불순도 감소량' 지정

In [1]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

* 결정 트리에서 사용할 min_impurity_decrease 매개변수의 값 후보를 입력
* fit() 메서드 : 그리드 서치 객체(gs)에서 min_impurity_decrease 값을 변경하며 총 5번 실행 
* GridSearchCV의 cv 매개변수 기본값 : 5, => 총 25개의 모델을 훈련
* n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수를 지정하면 속도를 빠르게 가능
    * 기본값 : 1, 
    * -1 : 시스템에 있는 모든 코어를 사용

In [2]:
import pandas as pd
import numpy as np

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
from sklearn.tree import DecisionTreeClassifier

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

* 가장 좋은 모델 출력

In [5]:
dt = gs.best_estimator_
print(dt)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)


In [6]:
print('score : {}'.format(dt.score(train_input, train_target)))
print('best hyper parameter : {}'.format(gs.best_params_))

score : 0.9615162593804117
best hyper parameter : {'min_impurity_decrease': 0.0001}


In [7]:
gs.cv_results_

{'mean_fit_time': array([0.00578251, 0.00517135, 0.00498385, 0.00420051, 0.00478292]),
 'std_fit_time': array([0.00096804, 0.00040961, 0.00062228, 0.0003877 , 0.00075315]),
 'mean_score_time': array([0.0003993 , 0.00040874, 0.00099735, 0.00058947, 0.00019908]),
 'std_score_time': array([0.00048906, 0.00050088, 0.00061961, 0.00048153, 0.00039816]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161

In [8]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [9]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


* hyper parameter 개수 늘려보자

In [10]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10),
          'min_samples_leaf': range(1, 25),
          }

In [11]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_leaf': range(1, 25),
                         'min_samples_split': range(2, 100, 10)})

In [12]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_leaf': 4, 'min_samples_split': 2}


In [13]:
gs.best_estimator_

DecisionTreeClassifier(max_depth=14, min_impurity_decrease=0.0004,
                       min_samples_leaf=4, random_state=42)

In [14]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695413489301844


#### **랜덤 서치**
* 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다
* 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다
* => 이럴 때 랜덤 서치(Random Search)를 사용
* 랜덤 서치 :
    * 매개변수 값의 목록을 전달하는 것이 아니라, 매개변수를 샘플링할 수 있는 **확률 분포** 객체를 전달 
---
* uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다 => 균등 분포에서 샘플링
    * randint : 정수값 추출
    * uniform : 실수값 추출

In [15]:
from scipy.stats import uniform, randint

* randint

In [16]:
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 8, 2, 4, 9, 4, 3, 2, 0, 7])

In [17]:
var, cnt = np.unique(rgen.rvs(1000), return_counts=True)

In [18]:
print(var)
print(cnt)
print(cnt.sum())

[0 1 2 3 4 5 6 7 8 9]
[107 113  95  83 101 102 108 101  96  94]
1000


* uniform

In [19]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.72882493, 0.77287154, 0.48252193, 0.79700301, 0.23303972,
       0.26759809, 0.59198961, 0.21693628, 0.87687678, 0.2274798 ])

In [20]:
np.unique(ugen.rvs(10), return_counts=True)

(array([0.04127429, 0.07099213, 0.1685766 , 0.26324087, 0.60579224,
        0.7918944 , 0.87682216, 0.89814266, 0.90619572, 0.91523543]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64))

---

In [21]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

* 위의 파라미터의 min_imputiry_decrease는 0.0001에서 0.001 사이의 실수값을 샘플링
* max_depth : 20에서 50 사이의 정수 
* min_sample_split : 2에서 25사이의 정수 
* min_samples_leaf : 1에서 25사이의 정수 샘플링 
---------
* RandomizedSearchCV 클래스를 사용
* n_iter : 샘플링의 횟수
* params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행, 최적의 매개변수 조합 탐색 
* 앞서 그리드 서치보다 휠씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있는 장점

In [22]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002047B1CFC40>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020479BBA310>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020475332790>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020479060CD0>},
                   random_state=42)

In [23]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [24]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [25]:
dt = rs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.8928227823744468
0.86
